# Importing libraries

In [12]:
import pathlib
import pandas as pd
import calendar
import numpy as np

from dash.dependencies import Input, Output
import plotly.express as px
import plotly.subplots as sp
import plotly.graph_objects as go
from dash import Dash, html, dcc

import requests
from io import StringIO
import os

In [13]:
dir =  os.getcwd()
FILE = dir + "/SQL/SQL_output.csv" 
RAIN = dir + "/Resources/2015_RainFall.csv"

In [14]:
rain = pd.read_csv(RAIN)
df = pd.read_csv(FILE)
df if df.isna().sum(axis=1).any() else print("No missing values") 
# Set the display options
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

No missing values


In [15]:
df.rename(columns={'pizza_type_id': 'pizza_flavor'}, inplace=True) # I think this name is more appropriate

# Splitting the date into month and days
df['date'] = pd.to_datetime(df['date'])
df['month'] = df['date'].dt.month
df['day'] = df['date'].dt.day
df['day_of_week'] = df['date'].dt.day_name()
day_order = ['Sunday', 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday']
df['hour'] = pd.to_datetime(df['time']).dt.hour


# Create 'order_id' column
df['order_id'] = df['month'].apply(lambda x: str(x).zfill(2)) + df['day'].apply(lambda x: str(x).zfill(2)) + df['time']

# Create 'multiple_orders' column, 1 if the order_id is duplicated, 0 if not
df['multiple_orders'] = df.duplicated('order_id').astype(int)

# Fixing typing errors in 'ingredients' column

df['ingredients'] = df['ingredients'].str.replace(', ', ',') # one space
df['ingredients'] = df['ingredients'].str.replace(',  ', ',') # two spaces after comma

# Perform one-hot encoding on the 'ingredients' column
ingredients_dummies = df['ingredients'].str.get_dummies(',')

# Add the new columns to the original DataFrame
df = pd.concat([df, ingredients_dummies], axis=1)

df = df.drop('ingredients', axis=1)


/var/folders/8n/frf4cdjd6k134p3nvgp4kf6c0000gn/T/ipykernel_7941/740048576.py:9: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['hour'] = pd.to_datetime(df['time']).dt.hour


In [16]:

# Group the data by hour and day
hourly_sum_quantity = df.groupby(['day_of_week', 'hour'])['quantity'].sum()




In [7]:
def scale_quantity(quantity):
    if quantity > 700:
        return 3
    elif quantity > 300:
        return 2
    elif quantity > 30:
        return 1
    else:
        return 0
    
def default(quantity):
    if quantity > 700:
        return 3
    elif quantity > 300:
        return 2
    elif quantity > 30:
        return 1
    else:
        return 0
hourly_sum_quantity
hourly_sum_quantity['employee_count'] = hourly_sum_quantity.apply(scale_quantity)


KeyError: 'quantity'

In [9]:
pd.DataFrame(hourly_sum_quantity)

quantity
day_of_week    hour                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         
Friday         11                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         

In [ ]:
bins = [0, 10, 300, 700, np.inf]
labels = [0, 1, 2, 3]
df['modified_employee_count'] = pd.cut(df['quantity_sum'], bins=bins, labels=labels)

In [13]:
hourly_sum_quantity['true_employee_count']

hour
    day_of_week  hour
Friday       11      2
     ...
Name: quantity, dtype: object

In [10]:
# Create the new heatmap using 'hour' and 'day_of_week' as axes and 'modified_employee_count' as values
heatmap_fig_new = px.density_heatmap(df, x='hour', y='day_of_week', z='modified_employee_count', color_continuous_scale='greens')

# Customize the color scale to go from green to black
heatmap_fig_new.update_coloraxes(colorscale='greens')

# Set axis labels and title
heatmap_fig_new.update_xaxes(title='Hour')
heatmap_fig_new.update_yaxes(title='Day of Week')
heatmap_fig_new.update_layout(title='Employee Count Based on Order Quantity')

heatmap_fig_new


In [20]:
results = df.groupby('month')['order_id'].count()
print (results)
months = list(range(1, 13))
days_in_month = [calendar.monthrange(2015, month)[1] for month in months]
print(days_in_month)
adjusted_quantity = results / days_in_month 
print (adjusted_quantity)

month
1     4156
2     3892
3     4186
4     4067
5     4239
6     4025
7     4301
8     4094
9     3819
10    3797
11    4185
12    3859
Name: order_id, dtype: int64
[31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
month
1     134.064516
2     139.000000
3     135.032258
4     135.566667
5     136.741935
6     134.166667
7     138.741935
8     132.064516
9     127.300000
10    122.483871
11    139.500000
12    124.483871
Name: order_id, dtype: float64


month
1     4156
2     3892
3     4186
4     4067
5     4239
6     4025
7     4301
8     4094
9     3819
10    3797
11    4185
12    3859
Name: order_id, dtype: int64

In [ ]:
dir =  os.getcwd()
FILE = dir + "/SQL/SQL_output.csv" 
RAIN = dir + "/Resources/2015_RainFall.csv"

PATH = pathlib.Path(__file__).parent
DATA_PATH = PATH.joinpath("data").resolve()


df = pd.read_csv(DATA_PATH.joinpath(FILE))
rain = pd.read_csv(DATA_PATH.joinpath(RAIN))

# Using Dash to create web layout

In [ ]:

app = Dash(__name__) 
server= app.server

# Define the app layout
app.layout = html.Div([
    html.H1("Python for Business Analytics - Jean Batista"),
    dcc.Tabs(id="tabs", value='tab-1', children=[
        dcc.Tab(label='Peak Hours', value='tab-1'),
        dcc.Tab(label='Weather Correlation', value='tab-2'),
        dcc.Tab(label='Variety', value='tab-3'),
        dcc.Tab(label='Sizes', value='tab-4'),
    ]),
    html.Div(id='content')
])


# Define the callback to update the content based on the selected tab
@app.callback(
    Output('content', 'children'),
    Input('tabs', 'value')
)

# Each tab displays correct graph on web

In [ ]:

def render_content(tab):
    if tab == 'tab-1':
        return html.Div([
            html.H2('Orders each hour, Day dropdown'),
            dcc.Dropdown(
                id='day-dropdown',
                options=[{'label': day, 'value': day} for day in day_order],
                value='Sunday',
                style={'margin-bottom': '10px'}  
            ),
            html.Div([
                dcc.Graph(
                    id='heatmap-graph',
                    figure=update_heatmap('Sunday'), 
                    style={'margin-bottom': '120px'}  
                ),
                html.Hr(style={'border-top': '6px solid rgba(0, 0, 0, 0.5)', 'margin': '10px 0'})  
            ], style={'margin-bottom': '120px'}),  
            html.H2('Days and Hours Descending'),
            dcc.Graph(
                id='treemap-graph',
                figure=update_treemap(),
                style={'margin-bottom': '250px'}  
            )
        ])
    elif tab == 'tab-2':
        return html.Div([
            html.H2('Rainfall and Orders'),
            dcc.Graph(
                id='subplot-graph',
                figure=update_subplot_graph()
            )
        ])
    elif tab == 'tab-3':
        return html.Div([
            html.H2('Pizza Types'),
            dcc.Dropdown(
                id='category-dropdown',
                options=[{'label': c, 'value': c} for c in df['category'].unique()],
                value=df['category'].unique()[0],
                style={'margin-bottom': '10px'}  
            ),
            dcc.Dropdown(
                id='pizza-flavor-dropdown',  # This will be populated based on the category selected
                style={'margin-bottom': '10px'}  
            ),
            dcc.Slider(
                id='month-slider',
                min=df['month'].min(),
                max=df['month'].max(),
                value=df['month'].min(),
                marks={str(month): str(month) for month in df['month'].unique()},
                step=None,
            ),
            html.Div(id='selected-pizza-stats'),  # This will display the selected pizza statistics
            dcc.Graph(
                id='scatterplot',
                figure={}  # To be updated with the callback
            )
        ])
    elif tab == 'tab-4':
        return html.Div([
            html.H2('Pizza Sizes'),
            dcc.Graph(
                id='pie-chart',
                figure=update_pie_chart()
            ),
            dcc.Graph(
                id='bar-chart',
                figure=update_bar_chart()
            )
        ])

# Connect dash components with figures and return visual

In [ ]:
@app.callback(
    Output('heatmap-graph', 'figure'),
    Input('day-dropdown', 'value')
)
 
# A callback is neccesary because it allows us to 
# update the graph based on the dropdown selection


# Heatmap

In [ ]:

def update_heatmap(day):
    filtered_df = df[df['day_of_week'] == day]
    pizza_type_counts = filtered_df['pizza_flavor'].value_counts()
    popular_pizza_types = pizza_type_counts.nlargest(15).index.tolist()
    filtered_df = filtered_df[filtered_df['pizza_flavor'].isin(popular_pizza_types)]
    filtered_df['hour'] = pd.to_numeric(filtered_df['hour'])  # Convert hour column to numeric
    filtered_df = filtered_df.sort_values('hour')  # Sort DataFrame by hour column
    heatmap_fig = px.density_heatmap(filtered_df, x='hour', y='pizza_flavor', z='quantity', color_continuous_scale='RdYlGn')
    heatmap_fig.update_layout(xaxis={'type': 'category'})  

    return heatmap_fig

# Rainfall

In [ ]:

def update_subplot_graph():
    results = df.groupby('month')['order_id'].count()
    months = list(range(1, 13))
    days_in_month = [calendar.monthrange(2015, month)[1] for month in months]
    adjusted_quantity = results / days_in_month 
    rainfall = rain.groupby('month').sum()

    # Create the subplot fig
    fig = sp.make_subplots(specs=[[{"secondary_y": True}]])
    fig.add_trace(go.Scatter(x=months, y=adjusted_quantity, name='Orders', line=dict(color='green')), secondary_y=False)
    fig.add_trace(go.Scatter(x=months, y=rainfall['rainfall'], name='Rainfall', line=dict(color='blue')), secondary_y=True)
    fig.update_layout(
        xaxis=dict(title='Month', dtick=1),
        yaxis=dict(
            title='Orders per day',
            titlefont=dict(color='black'),
            tickfont=dict(color='green')
        ),
        yaxis2=dict(
            title='Rainfall in inches',
            titlefont=dict(color='black'),
            tickfont=dict(color='blue'),
            anchor='x',
            overlaying='y',
            side='right'
        ),
        legend=dict(x=0, y=1.2, orientation='h')
    )

    return fig

# Pizza Types Histogram

In [ ]:
def update_histogram():
    sorted_df = df.sort_values('quantity', ascending=False)
    histogram_fig = px.histogram(sorted_df, x='pizza_flavor', y='quantity', color='category', barmode='group')
    
    return histogram_fig

# The Greek pie chart

In [ ]:
# I have the pie chart to evaluate if XL and XXL are viable options
def update_pie_chart():
    filtered_df = df[df['pizza_flavor'] == 'the_greek']
    size_quantity = filtered_df.groupby('size')['quantity'].sum().reset_index()

    pie_chart_fig = go.Figure(data=go.Pie(labels=size_quantity['size'], values=size_quantity['quantity']))

    pie_chart_fig.update_layout(title='Pizza Sizes Distribution for "The Greek"',
                                showlegend=True)

    return pie_chart_fig

# Sizes Bar chart

In [ ]:
# I've removed the greek, brie carre, big meats, and five cheese because they would represent outlier
# Because of their size options
def update_bar_chart():
    filtered_df_pie = df[df['pizza_flavor'] == 'the_greek']
    filtered_df_bar = df.copy()

    excluded_sizes = ['XL', 'XXL']
    filtered_df_bar = filtered_df_bar[~filtered_df_bar['size'].isin(excluded_sizes)]

    excluded_pizza_types = ['big_meat', 'brie_carre', 'five_cheese']
    filtered_df_bar = filtered_df_bar[~filtered_df_bar['pizza_flavor'].isin(excluded_pizza_types)]

    size_quantity_bar = filtered_df_bar.groupby('size')['quantity'].sum().reset_index().sort_values('quantity')

    bar_chart_fig = go.Figure(data=go.Bar(x=size_quantity_bar['size'], y=size_quantity_bar['quantity']))

    bar_chart_fig.update_layout(title='Summed Quantity for Each Pizza Size (Excluding XL, XXL, and Specific Types)',
                                xaxis_title='Size',
                                yaxis_title='Summed Quantity')

    return bar_chart_fig


# Run

In [ ]:

if __name__ == '__main__':
    app.run_server()

## Copy paste app.py to help me fix it tomorrow 

In [ ]:
""" import pathlib
import pandas as pd
import calendar

from dash.dependencies import Input, Output
import plotly.express as px
import plotly.subplots as sp
import plotly.graph_objects as go
from dash import Dash, html, dcc

import requests
from io import StringIO
import os

FILE = "/Users/jcarlito/Documents/GitHub/DA-Business-Analytics-2023/SQL/SQL_output.csv"
RAIN = "/Users/jcarlito/Documents/GitHub/DA-Business-Analytics-2023/Resources/2015_RainFall.csv"


PATH = pathlib.Path(__file__).parent
DATA_PATH = PATH.joinpath("data").resolve()


df = pd.read_csv(DATA_PATH.joinpath(FILE))
rain = pd.read_csv(DATA_PATH.joinpath(RAIN))

df.rename(columns={'pizza_type_id': 'pizza_flavor'}, inplace=True) # I think this name is more appropriate


# Splitting the date into month and days
df['date'] = pd.to_datetime(df['date'])
df['month'] = df['date'].dt.month
df['day'] = df['date'].dt.day
df['day_of_week'] = df['date'].dt.day_name()
day_order = ['Sunday', 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday']
df['hour'] = pd.to_datetime(df['time']).dt.hour


# Year doesnt matter since it doesnt change, dropped it

# Create 'order_id' column so we can check for multiple orders within same time
df['order_id'] = df['month'].astype(str) + df['day'].astype(str) + df['time'].astype(str)

# Create 'multiple_orders' column, 1 if the order_id is duplicated, 0 if not
df['multiple_orders'] = df.duplicated('order_id').astype(int)

# Fixing typing errors in 'ingredients' column

df['ingredients'] = df['ingredients'].str.replace(', ', ',') # one space
df['ingredients'] = df['ingredients'].str.replace(',  ', ',') # two spaces after comma

# Perform one-hot encoding on the 'ingredients' column
ingredients_dummies = df['ingredients'].str.get_dummies(',')

# Add the new columns to the original DataFrame
df = pd.concat([df, ingredients_dummies], axis=1)

df = df.drop('ingredients', axis=1)

app = Dash(__name__) 
server= app.server

# Define the app layout
app.layout = html.Div([
    html.H1("Python for Business Analytics - Jean Batista"),
    dcc.Tabs(id="tabs", value='tab-1', children=[
        dcc.Tab(label='Peak Hours', value='tab-1'),
        dcc.Tab(label='Weather Correlation', value='tab-2'),
        dcc.Tab(label='Variety', value='tab-3'),
        dcc.Tab(label='Sizes', value='tab-4'),
    ]),
    html.Div(id='content')
])
# Define the callback to update the content based on the selected tab
@app.callback(
    Output('content', 'children'),
    Input('tabs', 'value')
)

def render_content(tab):
    if tab == 'tab-1':
        return html.Div([
            html.H2('Orders each hour, Day dropdown'),
            dcc.Dropdown(
                id='day-dropdown',
                options=[{'label': day, 'value': day} for day in day_order],
                value='Sunday',
                style={'margin-bottom': '10px'}  
            ),
            html.Div([
                dcc.Graph(
                    id='heatmap-graph',
                    figure=update_heatmap('Sunday'), 
                    style={'margin-bottom': '120px'}  
                ),
                html.Hr(style={'border-top': '6px solid rgba(0, 0, 0, 0.5)', 'margin': '10px 0'})  
            ]) 
        ])
    elif tab == 'tab-2':
        return html.Div([
            html.H2('Rainfall and Orders'),
            dcc.Graph(
                id='subplot-graph',
                figure=update_subplot_graph()
            )
        ])
    elif tab == 'tab-3':
        return html.Div([
            html.H2('Pizza Types'),
            dcc.Graph(
                id='histogram',
                figure=update_histogram()
            )
        ])
    elif tab == 'tab-4':
        return html.Div([
            html.H2('Pizza Sizes'),
            dcc.Graph(
                id='pie-chart',
                figure=update_pie_chart()
            ),
            dcc.Graph(
                id='bar-chart',
                figure=update_bar_chart()
            )
        ])
    


@app.callback(
    Output('heatmap-graph', 'figure'),
    Input('day-dropdown', 'value')
)
def update_heatmap(day):
    filtered_df = df[df['day_of_week'] == day]
    pizza_type_counts = filtered_df['pizza_flavor'].value_counts()
    popular_pizza_types = pizza_type_counts.nlargest(15).index.tolist()
    filtered_df = filtered_df[filtered_df['pizza_flavor'].isin(popular_pizza_types)]
    filtered_df['hour'] = pd.to_numeric(filtered_df['hour'])  # Convert hour column to numeric
    filtered_df = filtered_df.sort_values('hour')  # Sort DataFrame by hour column
    heatmap_fig = px.density_heatmap(filtered_df, x='hour', y='pizza_flavor', z='quantity', color_continuous_scale='RdYlGn')
    heatmap_fig.update_layout(xaxis={'type': 'category'})  

    return heatmap_fig


def update_subplot_graph():
    results = df.groupby('month')['quantity'].sum()
    months = list(range(1, 13))
    days_in_month = [calendar.monthrange(2015, month)[1] for month in months]
    adjusted_quantity = results / days_in_month * 30
    rainfall = rain.groupby('month').sum()

    # Create the subplot fig
    fig = sp.make_subplots(specs=[[{"secondary_y": True}]])
    fig.add_trace(go.Scatter(x=months, y=adjusted_quantity, name='Orders', line=dict(color='green')), secondary_y=False)
    fig.add_trace(go.Scatter(x=months, y=rainfall['rainfall'], name='Rainfall', line=dict(color='blue')), secondary_y=True)
    fig.update_layout(
        xaxis=dict(title='Month', dtick=1),
        yaxis=dict(
            title='Orders',
            titlefont=dict(color='black'),
            tickfont=dict(color='green')
        ),
        yaxis2=dict(
            title='Rainfall in inches',
            titlefont=dict(color='black'),
            tickfont=dict(color='blue'),
            anchor='x',
            overlaying='y',
            side='right'
        ),
        legend=dict(x=0, y=1.2, orientation='h')
    )

    return fig

def update_histogram():
    sorted_df = df.sort_values('quantity', ascending=False)
    histogram_fig = px.histogram(sorted_df, x='pizza_flavor', y='quantity', color='category', barmode='group')
    
    return histogram_fig

# I have the pie chart to evaluate if XL and XXL are viable options
def update_pie_chart():
    filtered_df = df[df['pizza_flavor'] == 'the_greek']
    size_quantity = filtered_df.groupby('size')['quantity'].sum().reset_index()

    pie_chart_fig = go.Figure(data=go.Pie(labels=size_quantity['size'], values=size_quantity['quantity']))

    pie_chart_fig.update_layout(title='Pizza Sizes Distribution for "The Greek"',
                                showlegend=True)

    return pie_chart_fig

# I've removed the greek, brie carre, big meats, and five cheese because they would represent outlier
# Because of their size options
def update_bar_chart():
    filtered_df_pie = df[df['pizza_flavor'] == 'the_greek']
    filtered_df_bar = df.copy()

    excluded_sizes = ['XL', 'XXL']
    filtered_df_bar = filtered_df_bar[~filtered_df_bar['size'].isin(excluded_sizes)]

    excluded_pizza_types = ['big_meat', 'brie_carre', 'five_cheese']
    filtered_df_bar = filtered_df_bar[~filtered_df_bar['pizza_flavor'].isin(excluded_pizza_types)]

    size_quantity_bar = filtered_df_bar.groupby('size')['quantity'].sum().reset_index().sort_values('quantity')

    bar_chart_fig = go.Figure(data=go.Bar(x=size_quantity_bar['size'], y=size_quantity_bar['quantity']))

    bar_chart_fig.update_layout(title='Summed Quantity for Each Pizza Size (Excluding XL, XXL, and Specific Types)',
                                xaxis_title='Size',
                                yaxis_title='Summed Quantity')

    return bar_chart_fig


if __name__ == '__main__':
    app.run_server()"""
